In [ ]:
import sys
sys.path.append('../')

### Run the optimization

In [ ]:
import os

import numpy as np
from common import get_dataframe, evaluate_best_coin, objective

import mlflow
import optuna

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
mlflow.set_experiment('Training_LSTM')

# Normalize the data to the range [0, 1] to help the LSTM model converge faster

df = get_dataframe()

for coin in df.iloc[:, 1:]:
    data = np.array(df[coin]).reshape(-1, 1)

    # Create an Optuna study
    study_name = coin  # Unique identifier of the study.
    storage_name = "sqlite:///training.db"
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(study_name=coin, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, coin), n_trials=50)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(coin, data, best_params)
    except ValueError:
        print("No best hyperparameters found")